# Environment Setup
Datasetthese commands TODO: org

Push a Nvidia repo to your artifact repository - (create a new repo if not available)

The "new repo" would be us-central1-docker.pkg.dev/hybrid-vertex/merlin/ below

```
sudo docker pull nvcr.io/nvidia/merlin/merlin-hugectr:22.06
sudo docker tag nvcr.io/nvidia/merlin/merlin-hugectr:22.06 us-central1-docker.pkg.dev/hybrid-vertex/merlin/merlin-hugectr:22.06
sudo docker push us-central1-docker.pkg.dev/hybrid-vertex/merlin/merlin-hugectr:22.06
docker run  -d --gpus all --rm -it -p 8888:8888 -p 8797:8787 -p 8796:8786 --ipc=host --cap-add SYS_NICE us-central1-docker.pkg.dev/hybrid-vertex/merlin/merlin-hugectr:22.06 
jupyter-lab --allow-root --ip='0.0.0.0' --NotebookApp.token=''
git clone https://github.com/NVIDIA-Merlin/Merlin.git


docker run  -d --gpus all --rm -it -p 8888:8888 -p 8797:8787 -p 8796:8786 --ipc=host --cap-add SYS_NICE nvcr.io/nvidia/merlin/merlin-hugectr:22.06 
```


# Training with Merlin

Spotify example

The goal of this notebook is todo

based heavily on [this](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb) NVIDIA resource


Data is loaded from BQ - to parquet using the job configuration below

```
BUCKET = 'gs://spotify-builtin-2t'
PROJECT = 'hybrid-vertex'
DATASET_ID = 'spotify_train_3'
TABLE = 'train_flatten'
TABLE_SMALL = 'train_json_export_table_small'
LOCATION = 'us-central1'

from google.cloud import bigquery
client = bigquery.Client()

destination_uri = f"{BUCKET}/train_data_parquet/*.snappy.parquet"
dataset_ref = bigquery.DatasetReference(PROJECT, DATASET_ID)
table_ref = dataset_ref.table(TABLE)
job_config = bigquery.job.ExtractJobConfig()
job_config.destination_format = bigquery.DestinationFormat.PARQUET
extract_job = client.extract_table(
    table_ref,
    destination_uri,
    job_config=job_config,
    # Location must match that of the source table.
    location=LOCATION,
)  # API request
extract_job.result()  # Waits for job to complete.
```


Info on the data

* track_uri_seed counts: 2249561
* artist_uri_seed counts: 294110
* album_uri_seed counts: 730377

In [1]:
# !pip install gcsfs gsutil tensorflow

In [1]:
import os
import nvtabular as nvt
from time import time

# disable INFO and DEBUG logging everywhere
import logging

logging.disable(logging.WARNING)

from nvtabular.ops import (
    Categorify,
    TagAsUserID,
    TagAsItemID,
    TagAsItemFeatures,
    TagAsUserFeatures,
    AddMetadata,
    ListSlice
)
import nvtabular.ops as ops

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
import tensorflow as tf

# for running this example on CPU, comment out the line below
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

2022-08-15 16:37:04.548321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-15 16:37:04.602508: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-15 16:37:04.602856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:952] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-15 16:37:05.003388: I tensorflow/core/platform/cpu_feature_guard.cc:152] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate 

### Install package to access gcs data

In [ ]:
BUCKET = 'gs://spotify-builtin-2t'
PROJECT = 'hybrid-vertex'

LOCATION = 'us-central1'
# 000000000000.snappy.parquet - example record output
train = nvt.Dataset(f"{BUCKET}/train_data_parquet/0000000000**.snappy.parquet")
valid = nvt.Dataset(f"{BUCKET}/validation_data_parquet/00000000000*.snappy.parquet")
MAX_PADDING = 375
# train = nvt.Dataset(f"{BUCKET}/train_data_parquet/*.snappy.parquet")
# valid = nvt.Dataset(f"{BUCKET}/validation_data_parquet/*.snappy.parquet")

#### Use `cardinality_memory_limit` or `max_size` to ensure lookups fit in memory 

```
cardinality_memory_limit: int or str, default None
    Upper limit on the "allowed" memory usage of the internal DataFrame and Table objects
    used to store unique categories. By default, this limit is 12.5% of the total memory.
    Note that this argument is meant as a guide for internal optimizations and UserWarnings
    within NVTabular, and does not guarantee that the memory limit will be satisfied.
```

In [ ]:
# ?Categorify

In [ ]:
item_id = ["track_uri_can"] >> Categorify(dtype="int32") >> ops.TagAsItemID() 
playlist_id = ["pid_pos_id"] >> Categorify(dtype="int32") >> TagAsUserID() 


item_features_cat = ['artist_name_can',
        'track_name_can',
        'artist_genres_can',
    ]

item_features_cont = [
        'duration_ms_can',
        'track_pop_can',
        'artist_pop_can',
        'artist_followers_can',
    ]

playlist_features_cat = [
        'artist_name_seed_track',
        'artist_uri_seed_track',
        'track_name_seed_track',
        'track_uri_seed_track',
        'album_name_seed_track',
        'album_uri_seed_track',
        'artist_genres_seed_track',
        'description_pl',
        'name',
        'collaborative',
    ]

playlist_features_cont = [
        'duration_seed_track',
        'track_pop_seed_track',
        'artist_pop_seed_track',
        'artist_followers_seed_track',
        'duration_ms_seed_pl',
        'n_songs_pl',
        'num_artists_pl',
        'num_albums_pl',
    ]

#subset of features to be tagged
seq_feats_cont = [
        'duration_ms_songs_pl',
        'artist_pop_pl',
        'artists_followers_pl',
        'track_pop_pl',
    ]

seq_feats_cat = [
        'artist_name_pl',
        # 'track_uri_pl',
        'track_name_pl',
        'album_name_pl',
        'artist_genres_pl',
        # 'pid_pos_id', 
        # 'pos_pl'
    ]

CAT = playlist_features_cat + item_features_cat
CONT = item_features_cont + playlist_features_cont

item_feature_cat_node = item_features_cat >> nvt.ops.FillMissing()>> Categorify(dtype="int32") >> TagAsItemFeatures()

item_feature_cont_node =  item_features_cont >> nvt.ops.FillMissing() >>  nvt.ops.Normalize() >> TagAsItemFeatures()

playlist_feature_cat_node = playlist_features_cat >> nvt.ops.FillMissing() >> Categorify(dtype="int32") >> TagAsUserFeatures() 

playlist_feature_cont_node = playlist_features_cont >> nvt.ops.FillMissing() >>  nvt.ops.Normalize() >> TagAsUserFeatures()

playlist_feature_cat_seq_node = seq_feats_cat >> nvt.ops.FillMissing() >> Categorify(dtype="int32") >> ListSlice(MAX_PADDING, pad=True, pad_value=0) >> TagAsUserFeatures() >> nvt.ops.AddTags(Tags.SEQUENCE) 

playlist_feature_cont_seq_node = seq_feats_cont >> nvt.ops.FillMissing() >>  nvt.ops.Normalize() >>  >> ListSlice(MAX_PADDING, pad=True, pad_value=0) >> TagAsUserFeatures() >> nvt.ops.AddTags(Tags.SEQUENCE)

In [ ]:
# define a workflow
output = playlist_id + item_id \
+ item_feature_cat_node \
+ item_feature_cont_node \
+ playlist_feature_cat_node \
+ playlist_feature_cont_node \
+ playlist_feature_cont_seq_node \
+ playlist_feature_cat_seq_node 


workflow = nvt.Workflow(output)

In [ ]:
output.graph

In [29]:
import pandas as pd

# Don't truncate text fields in the display
pd.set_option("display.max_colwidth", None)

valid.to_ddf().head(1)

,track_uri_can,artist_name_can,track_name_can,artist_genres_can,duration_ms_can,track_pop_can,artist_pop_can,artist_followers_can,artist_name_seed_track,artist_uri_seed_track,...,num_artists_pl,num_albums_pl,duration_ms_songs_pl,artist_pop_pl,artists_followers_pl,track_pop_pl,artist_name_pl,track_name_pl,album_name_pl,artist_genres_pl
0,3027,414,2693,374,-0.121892,-1.281445,0.115315,-0.417949,408,408,...,-0.957475,-0.892232,"[-0.29044408730033605, -0.17623619889540684, -0.5252146594871301, -0.20588276048111134, 0.907880912201834, 0.13634287219386812, 0.9252708166134486, -0.0794182149020923, 0.351005938490358, 0.21063770453804329, -0.43829258794904374, 1.3148760467855816, -0.6456400906690611]","[1.110541625408202, 0.6034335315430215, 0.6034335315430215, 0.6034335315430215, 0.6034335315430215, 0.6034335315430215, 0.5470881877802237, 0.5470881877802237, 0.5470881877802237, 0.5470881877802237, 0.09632543767784112, 0.09632543767784112, 0.09632543767784112]","[2.135336145609191, -0.05124952229509556, -0.05124952229509556, -0.05124952229509556, -0.05124952229509556, -0.05124952229509556, -0.3468518973364721, -0.3468518973364721, -0.3468518973364721, -0.3468518973364721, -0.4237680395250553, -0.4237680395250553, -0.4237680395250553]","[-1.2874509693596279, 1.3175559644415924, 0.9878082513021975, 1.3175559644415924, 0.8559091660464395, -1.2874509693596279, -1.2874509693596279, -1.2874509693596279, -1.2874509693596279, -1.2874509693596279, 0.7240100807906815, 0.8559091660464395, -1.2874509693596279]","[21, 104, 104, 104, 104, 104, 247, 247, 247, 247, 383, 383, 383, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[631, 41, 836, 852, 1384, 563, 163, 258, 1201, 7589, 5906, 678, 2781, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[466, 83, 83, 760, 83, 1371, 128, 128, 128, 128, 3364, 593, 2683, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[18, 122, 122, 122, 122, 122, 254, 254, 254, 254, 336, 336, 336, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


Now set the output buckets for the validation and training

In [8]:
output_path = os.path.join(BUCKET, "merlin-processed")
output_train_dir = os.path.join(output_path, 'train/')
output_valid_dir = os.path.join(output_path, 'valid/')
output_workflow_dir = os.path.join(output_path, 'workflow/')


print(f"Train data dir: {output_train_dir}\nValid data dir: {output_valid_dir}")

Train data dir: gs://spotify-builtin-2t/merlin-processed/train/
Valid data dir: gs://spotify-builtin-2t/merlin-processed/valid/


## Load and preprocess training data back to Parquet GCS
helpful resources https://cloud.google.com/compute/docs/gpus/monitor-gpus

Number of files in input bucket
```
jwortz-macbookpro:merlin-workflow-download jwortz$ gsutil ls -lR gs://spotify-builtin-2t/train_data_parquet | tail -n 1
TOTAL: 5000 objects, 326467491281 bytes (304.05 GiB)

jwortz-macbookpro:merlin-workflow-download jwortz$ gsutil ls -lR gs://spotify-builtin-2t/validation_data_parquet | tail -n 1
TOTAL: 152 objects, 16521928490 bytes (15.39 GiB)
```


In [8]:
%%time

time_preproc = 0
time_preproc_start = time()

workflow.fit_transform(train).to_parquet(output_path=output_train_dir, 
                                         shuffle=nvt.io.Shuffle.PER_PARTITION,
                                         cats=CAT,
                                         conts=CONT, 
                                         output_files = 50
                                         ) #preserve_files=True keeps the original file sharding


time_preproc += time()-time_preproc_start

/usr/local/lib/python3.8/dist-packages/cudf/core/frame.py:384: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


CPU times: user 4min 50s, sys: 2min 22s, total: 7min 12s
Wall time: 13min 13s


In [9]:
#save the workflow to disk after it has been fit
workflow.save(os.path.join(output_workflow_dir,'2t-spotify-workflow'))

#locally for demo too
workflow.save('2t-spotify-workflow')

In [10]:
schema = workflow.output_schema

schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension
0,track_uri_can,"(Tags.ITEM_ID, Tags.CATEGORICAL, user_item, Tags.ITEM)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.track_uri_can.parquet,0.0,258189.0,track_uri_can,258189.0,512.0
1,artist_name_can,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.artist_name_can.parquet,0.0,57696.0,artist_name_can,57696.0,512.0
2,track_name_can,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.track_name_can.parquet,0.0,196107.0,track_name_can,196107.0,512.0
3,artist_genres_can,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.artist_genres_can.parquet,0.0,20511.0,artist_genres_can,20511.0,416.0
4,duration_ms_can,"(Tags.CONTINUOUS, Tags.ITEM)",float64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,track_pop_can,"(Tags.CONTINUOUS, Tags.ITEM)",float64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,artist_pop_can,"(Tags.CONTINUOUS, Tags.ITEM)",float64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,artist_followers_can,"(Tags.CONTINUOUS, Tags.ITEM)",float64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,artist_name_seed_track,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.artist_name_seed_track.parquet,0.0,57557.0,artist_name_seed_track,57557.0,512.0
9,artist_uri_seed_track,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.artist_uri_seed_track.parquet,0.0,58165.0,artist_uri_seed_track,58165.0,512.0


## Load and preprocess validation

In [11]:
%%time

time_preproc_start = time()
wf_valid_op = workflow.transform(valid).to_parquet(output_path=output_valid_dir, 
                                         shuffle=nvt.io.Shuffle.PER_PARTITION,
                                         cats=CAT,
                                         conts=CONT,
                                         output_files=10
                                         )


time_preproc += time()-time_preproc_start

CPU times: user 9.27 s, sys: 4.77 s, total: 14 s
Wall time: 31.3 s


In [163]:
nvtdata = pd.read_parquet(output_train_dir+'part_0.parquet')
nvtdata['track_name_pl'].head(3) #should see padded lists

0                                                                                                                                                                                                                                     [1246, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]
1                                                                                                                                                                             [303447, 1248, 12045, 57775, 398, 8623, 5034, 6138, 19477, 1340, 4382, 374, 1110, 927, 1145, 5400, 4524, 6739, 6027, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [161]:
nvtdata = pd.read_parquet(output_train_dir+'part_0.parquet')
nvtdata['artist_name_seed_track'].head(10)

0      21
1    1043
2    1055
3     396
4     445
5    2345
6    2969
7     497
8    1029
9    2124
Name: artist_name_seed_track, dtype: int32

In [162]:
len(nvtdata)

24680

In [ ]:
embeddings = ops.get_embedding_sizes(workflow)
embeddings

In [ ]:
print([embeddings[0][x] for x in item_feature_cat_node.output_columns.names])

In [ ]:
playlist_feature_cont_node.output_columns.names

# Load the processed data into a Merlin Dataset and inspect the transforms

Now that ETL is over, the workflow is saved and data is processed to the `output_path`

In [ ]:
# !gcloud set project-id hybrid-vertex

In [ ]:
!gsutil cp -r {output_workflow_dir} .

In [179]:
# load back the workflow and schema
# spotify-builtin-2t/merlin-processed/workflow/2t-spotify-workflow
workflow = nvt.Workflow.load("2t-spotify-workflow")
schema = workflow.output_schema
embeddings = ops.get_embedding_sizes(workflow)
embeddings

{'track_uri_can': (258189, 512),
 'artist_name_can': (57696, 512),
 'track_name_can': (196107, 512),
 'artist_genres_can': (20511, 416),
 'artist_name_seed_track': (57557, 512),
 'artist_uri_seed_track': (58165, 512),
 'track_name_seed_track': (195896, 512),
 'track_uri_seed_track': (257894, 512),
 'album_name_seed_track': (112919, 512),
 'album_uri_seed_track': (134740, 512),
 'artist_genres_seed_track': (20431, 415),
 'description_pl': (11574, 302),
 'name': (72117, 512),
 'collaborative': (3, 16),
 'artist_name_pl': (211490, 512),
 'track_name_pl': (1086902, 512),
 'album_name_pl': (426100, 512),
 'artist_genres_pl': (34844, 512)}

In [10]:
from merlin.models.utils.example_utils import workflow_fit_transform

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset as MerlinDataset

train = MerlinDataset(output_train_dir + "/*.parquet", schema=schema, part_size="500MB")
valid = MerlinDataset(output_valid_dir + "/*.parquet", schema=schema, part_size="500MB")

#look at output
schema = train.schema

In [ ]:
# from merlin.models.tf.blocks.core.transformations import AsSparseFeatures

# ?AsSparseFeatures

In [11]:
#removing sequence features for now to train the 2t model

two_t_schema = schema.select_by_tag([Tags.ITEM_ID, Tags.ITEM, Tags.USER, Tags.USER_ID])
two_t_schema_seq = schema.select_by_tag([Tags.SEQUENCE])
non_seq_col_names = list(set(two_t_schema.column_names) - set(two_t_schema_seq.column_names))
non_seq_col_names
# two_t_schema = [x for x in two_t_schema_seq.column_names]
two_t_schema = two_t_schema[non_seq_col_names]
two_t_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension
0,name,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.name.parquet,0.0,72117.0,name,72117.0,512.0
1,description_pl,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.description_pl.parquet,0.0,11574.0,description_pl,11574.0,302.0
2,artist_uri_seed_track,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.artist_uri_seed_track.par...,0.0,58165.0,artist_uri_seed_track,58165.0,512.0
3,track_uri_seed_track,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.track_uri_seed_track.parquet,0.0,257894.0,track_uri_seed_track,257894.0,512.0
4,artist_name_can,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.artist_name_can.parquet,0.0,57696.0,artist_name_can,57696.0,512.0
5,duration_ms_seed_pl,"(Tags.CONTINUOUS, Tags.USER)",float64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,album_uri_seed_track,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.album_uri_seed_track.parquet,0.0,134740.0,album_uri_seed_track,134740.0,512.0
7,artist_followers_can,"(Tags.CONTINUOUS, Tags.ITEM)",float64,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,artist_genres_seed_track,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.artist_genres_seed_track....,0.0,20431.0,artist_genres_seed_track,20431.0,415.0
9,track_name_seed_track,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,.//categories/unique.track_name_seed_track.par...,0.0,195896.0,track_name_seed_track,195896.0,512.0


In [77]:
two_t_schema.column_names

['collaborative',
 'artist_uri_seed_track',
 'artist_genres_can',
 'artist_followers_seed_track',
 'artist_pop_can',
 'track_pop_can',
 'n_songs_pl',
 'description_pl',
 'duration_ms_seed_pl',
 'num_albums_pl',
 'track_uri_can',
 'album_uri_seed_track',
 'duration_seed_track',
 'track_uri_seed_track',
 'duration_ms_can',
 'track_pop_seed_track',
 'track_name_seed_track',
 'artist_name_seed_track',
 'artist_pop_seed_track',
 'name',
 'artist_genres_seed_track',
 'track_name_can',
 'artist_followers_can',
 'album_name_seed_track',
 'num_artists_pl',
 'artist_name_can']

Direct quote from [here](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb)

### Negative sampling
Many datasets for recommender systems contain implicit feedback with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. To be able to learn from implicit feedback, we use the general (and naive) assumption that the interacted items are more relevant for the user than the non-interacted ones. In Merlin Models we provide some scalable negative sampling algorithms for the Item Retrieval Task. In particular, we use in this example the in-batch sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Building the Model
Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling here and here), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [ ]:
# format embeddings
embeddings_all = embeddings

emb_dims = {}
for k in list(embeddings_all.keys()):
    emb_dims.update({k: embeddings_all[k][1]})
emb_dims

In [13]:
model = mm.TwoTowerModel(
    two_t_schema,
    query_tower=mm.MLPBlock([1024,512,256], no_activation_last_layer=True),
    item_tower=mm.MLPBlock([1024,512,256], no_activation_last_layer=True),
    samplers=[mm.InBatchSampler()],
    embedding_options=mm.EmbeddingOptions(infer_embedding_sizes=True),
)

Let's explain the parameters in the TwoTowerModel():

* no_activation_last_layer: when set True, no activation is used for top hidden layer. Learn more here.
* infer_embedding_sizes: when set True, automatically defines the embedding dimension from the feature cardinality in the schema

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

* **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

* **Recall@10:** Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

We need to initialize the dataloaders.

In [14]:
%%time

model.compile(optimizer="adam", run_eagerly=False, metrics=[mm.RecallAt(1), mm.RecallAt(10), mm.NDCGAt(10)])
model.fit(train, validation_data=valid, batch_size=2048, epochs=3)

2022-07-18 19:58:19.062494: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Epoch 1/3
607/607 [==============================] - 98s 140ms/step - loss: 6.0359 - recall_at_1: 0.0897 - recall_at_10: 0.2405 - ndcg_at_10: 0.1586 - regularization_loss: 0.0000e+00 - val_loss: 5.3199 - val_recall_at_1: 0.1587 - val_recall_at_10: 0.3479 - val_ndcg_at_10: 0.2471 - val_regularization_loss: 0.0000e+00
Epoch 2/3
607/607 [==============================] - 87s 141ms/step - loss: 4.8612 - recall_at_1: 0.1915 - recall_at_10: 0.3979 - ndcg_at_10: 0.2873 - regularization_loss: 0.0000e+00 - val_loss: 5.2904 - val_recall_at_1: 0.1694 - val_recall_at_10: 0.3640 - val_ndcg_at_10: 0.2598 - val_regularization_loss: 0.0000e+00
Epoch 3/3
607/607 [==============================] - 86s 140ms/step - loss: 4.1133 - recall_at_1: 0.2687 - recall_at_10: 0.5016 - ndcg_at_10: 0.3777 - regularization_loss: 0.0000e+00 - val_loss: 5.9280 - val_recall_at_1: 0.1541 - val_recall_at_10: 0.3485 - val_ndcg_at_10: 0.2439 - val_regularization_loss: 0.0000e+00
CPU times: user 4min 29s, sys: 1min 15s, total

#### Save query model

In [16]:
artifact_path = os.path.join(output_path, 'spotify-2t-query-model')
model.save(artifact_path) #saves keras model

In [90]:
# !pip install google-cloud-aiplatform

### Save embeddings to GCS for each song - this will become our Matching Engine index

In [ ]:
from merlin.models.utils.dataset import unique_rows_by_features

item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
    .compute()
    .reset_index(drop=True)
)

item_embs = model.item_embeddings(
    MerlinDataset(item_features, schema=schema), batch_size=1024
)
item_embs_df = item_embs.compute(scheduler="synchronous")

In [ ]:
item_features

In [ ]:
drop_cols = schema.select_by_tag([Tags.ITEM]).column_names
drop_cols.remove('track_uri_can')
drop_cols

In [127]:
# select only item_id together with embedding columns
item_embeddings = item_embs_df.drop(
    columns=drop_cols
)

In [128]:
item_embeddings.head()

,track_uri_can,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,1,0.623825,0.245726,-0.923404,-0.495237,-0.840502,-0.977298,-0.327044,0.310249,-1.522218,...,-0.213952,-0.492551,-0.596096,0.251824,-0.965978,-1.143388,0.932237,-0.862994,-0.625651,-0.067532
1,2,0.567377,0.256723,-0.915077,-0.438709,-0.907190,-0.689214,-0.379282,0.350286,-1.393230,...,-0.156716,-0.592479,-0.598374,0.366744,-1.100182,-1.081851,0.710624,-1.094165,-0.815016,0.097025
2,3,0.536450,0.568170,-0.902210,-0.963018,-0.611129,-0.233710,-0.526880,0.461254,-1.427661,...,-0.478623,-0.713327,-1.030612,0.589793,-0.972196,-0.846859,0.542413,-1.118448,-0.844330,0.358904
3,4,0.846645,0.470713,-0.861141,-0.882409,-1.056704,-0.586758,-0.169602,1.023548,-1.511723,...,-0.527605,-0.678308,-0.619097,0.431517,-0.869154,-1.231006,0.697403,-1.047519,-0.782201,0.226161
4,5,0.853122,0.479180,-0.738377,-0.592910,-1.354706,-0.164263,-0.449048,0.573087,-1.297430,...,-0.212778,-0.709860,-0.886499,0.092986,-0.910573,-1.079607,0.512421,-1.460375,-0.721887,0.147679


Note that these item ids are encoded values, not the raw original values. We will eventually create the reverse dictionary lookup functionality to be able to map these encoded item ids to their original raw ids with one-line of code. But if you really want to do it now, you can easily map these ids to their original values using the `unique.item_id.parquet` file stored in the `categories` folder.

That's it! You finished deploying a multi-stage Recommender Systems on Triton Inference Server using Merlin framework.

In [154]:
item_data.track_uri_can[0]

<NA>

In [130]:
item_embeddings['track_uri_can']

0              1
1              2
2              3
3              4
4              5
           ...  
258183    258184
258184    258185
258185    258186
258186    258187
258187    258188
Name: track_uri_can, Length: 258188, dtype: int32

In [153]:
item_data = pd.read_parquet('categories/unique.track_uri_can.parquet')

# named_embeddings = item_embeddings.to_replace(lambda x: item_data.track_uri_can[x.index])
# named_embeddings
lookup_dict = dict(item_data['track_uri_can'])


item_embeddings_pd = item_embeddings.to_pandas()

named_embeddings = item_embeddings_pd

named_embeddings['track_uri_can'] = named_embeddings['track_uri_can'].apply(lambda l: lookup_dict[l])

named_embeddings

# for k in item_embeddings.keys():
#     if k != 'track_uri_can':
#         print(lookup_dict[int(k)])
#     else:
#         pass

    
# item_embeddings.reset_index()

# item_embeddings
# # named_embeddings = item_embeddings.replace(lookup_dict)

# item_embeddings['str_lookup']=item_embeddings['track_uri_can']

# # named_embeddings

,track_uri_can,0,1,2,3,4,5,6,7,8,...,246,247,248,249,250,251,252,253,254,255
0,spotify:track:7KXjTSCq5nL1LoYtL7XAwS,0.623825,0.245726,-0.923404,-0.495237,-0.840502,-0.977298,-0.327044,0.310249,-1.522218,...,-0.213952,-0.492551,-0.596096,0.251824,-0.965978,-1.143388,0.932237,-0.862994,-0.625651,-0.067532
1,spotify:track:3a1lNhkSLSkpJE4MSHpDu9,0.567377,0.256723,-0.915077,-0.438709,-0.907190,-0.689214,-0.379282,0.350286,-1.393230,...,-0.156716,-0.592479,-0.598374,0.366744,-1.100182,-1.081851,0.710624,-1.094165,-0.815016,0.097025
2,spotify:track:1xznGGDReH1oQq0xzbwXa3,0.536450,0.568170,-0.902210,-0.963018,-0.611129,-0.233710,-0.526880,0.461254,-1.427661,...,-0.478623,-0.713327,-1.030612,0.589793,-0.972196,-0.846859,0.542413,-1.118448,-0.844330,0.358904
3,spotify:track:7yyRTcZmCiyzzJlNzGC9Ol,0.846645,0.470713,-0.861141,-0.882409,-1.056704,-0.586758,-0.169602,1.023548,-1.511723,...,-0.527605,-0.678308,-0.619097,0.431517,-0.869154,-1.231006,0.697403,-1.047519,-0.782201,0.226161
4,spotify:track:7BKLCZ1jbUBVqRi2FVlTVw,0.853122,0.479180,-0.738377,-0.592910,-1.354706,-0.164263,-0.449048,0.573087,-1.297430,...,-0.212778,-0.709860,-0.886499,0.092986,-0.910573,-1.079607,0.512421,-1.460375,-0.721887,0.147679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258183,spotify:track:7zyYROflk5MgxcMjpNrCqT,-0.285365,1.504289,-0.979205,-0.131530,-0.630109,-0.099629,-1.077468,-0.352738,-0.360940,...,-0.015841,-0.065987,-0.842878,0.956247,-0.168823,-0.789443,-0.006043,0.523338,-0.462480,0.504848
258184,spotify:track:7zyYmIdjqqiX6kLryb7QBx,0.062755,0.671345,-0.021377,-0.274562,-0.203041,-0.340279,-0.918186,0.424996,-0.140819,...,-0.412153,-0.376804,-0.904229,1.005881,-1.316041,-1.077826,0.899121,-1.080571,-0.126850,0.614000
258185,spotify:track:7zyicrdArhk1b51MjwZowi,0.628695,1.027862,0.551159,-0.713700,-1.491989,0.209310,-1.472905,1.588813,-0.055847,...,-0.258581,-1.224497,-0.887948,0.761131,-0.040258,-0.814650,0.187362,-1.608888,-0.133768,0.925695
258186,spotify:track:7zyo93bLysAjU08Enina9r,0.378318,0.896169,0.167172,0.005971,-1.286640,-0.116012,-1.978139,1.099975,0.279388,...,-0.052871,-1.034225,0.060401,1.080133,-0.268736,-1.199556,0.081056,-1.001086,-0.627435,0.495291


In [155]:
len(item_embeddings)

258188

### Finally, save the embeddings to gcs
todo - integrate workflow to map original track_uri

In [156]:
item_embeddings.to_parquet(os.path.join(output_path,'song-embeddings'))